In [1]:
from numpy import binary_repr
import numpy as np
import logging
import random
from scipy.sparse import csr_matrix

In [2]:

class inst:
    def __init__(self):
        logging.debug("Instruction generator created")
        
### Generates load instructions for GCN-Accelerator

    def load(self,  dep = '0000', id = 'col', sram_offset = 0, dram_offset = 0, xsize = 0, ysize = 0):
        idMap = {'col': '000','row': '001', 'val': '010', 'den': '011', 'out': '100'}
        op = '00'
        logging.debug(f"Creating load with \ndep = {dep}\nid = {id}\nsram offset = {sram_offset}\ndram offset = {dram_offset}\
        \nxsize = {xsize}\nysize = {ysize}")
        inst = op[::-1] + dep[::-1] + idMap.get(id)[::-1] + binary_repr(dram_offset, 32)[::-1]+ binary_repr(sram_offset, 16)[::-1]  
        logging.debug(f"inst[63:0] = {inst}")
        inst =  inst + binary_repr(xsize, 7)[::-1] + binary_repr(ysize, 0)[::-1]
        inst = inst + '0'*(128-len(inst))
        logging.debug(f"inst[127:0] = {inst}")
        return inst

    def spMM(self,  dep = '0000', sram_offset_col = 0, sram_offset_ptr = 0, sram_offset_den = 0, xsize = 0, ysize = 0):
        op = '10'
        logging.debug(f"Creating load with \ndep = {dep}\nid = {id}\nsram_offset_col = {sram_offset_col}\nsram_offset_ptr = {sram_offset_ptr}\
        \nsram_offset_den = {sram_offset_den}\nxsize = {xsize}\nysize = {ysize}")
        inst = op[::-1] + dep[::-1] + binary_repr(sram_offset_col, 16)[::-1] + binary_repr(sram_offset_ptr, 16)[::-1] + binary_repr(sram_offset_den, 16)[::-1]
        inst =  inst + binary_repr(xsize, 7)[::-1] + binary_repr(ysize, 7)[::-1]
        inst = inst + '0'*(128-len(inst))
        logging.debug(f"inst[127:0] = {inst}")
        return inst

    def randLoad(self, count = 1):
        instSeq = ""
        for num in range(count):
            instSeq = instSeq + "000000000"
            for i in range (119):
                instSeq = instSeq + str(random.randint(0,1))
        return instSeq
    
    def randSpMM(self, count = 1):
        instSeq = ""
        for num in range(count):
            instSeq = instSeq + "010000000"
            for i in range (119):
                instSeq = instSeq + str(random.randint(0,1))
        return instSeq

In [4]:
class data:
            
    ### Generates data for GCN-Accelerator
    def __init__(self):
        logging.debug("Data generator created")
    def matrixToBinary(self, A):
        string_A = ''
        for i in A:
            for j in i:
                string_A = string_A + binary_repr(j, 32)[::-1]
        return string_A
    def arrayToBinary(self, A):
        string_A = ''
        for i in A:
                string_A = string_A + binary_repr(i, 32)[::-1]
        return string_A


In [5]:
np.random.seed(0)

In [13]:
np.random.choice([3,-3],p=[0.4,0.6], size = (2,2))

array([[-3, -3],
       [ 3, -3]])

In [14]:
def randSP(spDim, denDim):
    lis = [0]
    lis2 = [0.8]
    for i in range(100):
        lis.append(np.random.randint(100))
        lis2.append((1-lis2[0])/100)
    I = np.random.choice(lis,p=lis2, size = (16,16))
    print(f"Nonzero is {np.count_nonzero(I)}")
    print(f"density is {np.count_nonzero(I)/I.size}")
    Sparse = csr_matrix(I)
    val = Sparse.data
    col = Sparse.indices
    row = Sparse.indptr
    W = np.random.randint(100, size = denDim)
    O = np.matmul(I,W)
    return ((val,col,row), W, O)
((val,col,row),den,O) = randSP(spDim = (16,16), denDim = (16,4))
print(f"\nvalues array is {val}")
print(f"\n row array is {row}")
print(f"\n col array is {col}")
print(f"\n den array is {den}")
print(O)

Nonzero is 63
density is 0.24609375

values array is [73 37 72 23 40 21 36 72 73 87 57 93 67 10 76 21  8 73 29 40 15 14 51 54
 19 45 73 13 16 39 76  1 75 10  7 75 15 63 18 16  1 79 16 73 14 75 37 67
 26 57 61 58 72 32 79 79 37  6 19  7 58 88 39]

 row array is [ 0  4  8 15 18 21 24 29 36 38 43 45 50 53 57 59 63]

 col array is [ 3  6 11 15  1  2  9 11  0  2  4  5 10 11 15  0 11 13 10 12 13  2  3  9
  0  4  5 14 15  3  5  7  8  9 12 13  2  4  0  5  8 10 14 10 14  0  6  7
 13 14  4 10 11  3 10 13 14  4  9  5  6  8 12]

 den array is [[59 17 20  2]
 [37 80 28 37]
 [21 58 33  4]
 [90 90 44  9]
 [50 25 12 85]
 [70 67 99 70]
 [78 30 15 26]
 [54 60 47 13]
 [76 78 16 55]
 [65 55  7 47]
 [47 14 59 28]
 [17  2 89 37]
 [28  0 23 99]
 [ 3 24 24 10]
 [41 54  2 55]
 [84 77 41 75]]
[[12612  9595 11118  6008]
 [ 5485  6542  8473  5920]
 [25197 20753 22181 19795]
 [ 1594  2125  2884  1068]
 [ 2528   766  2991  4922]
 [ 8394  8372  3084  3053]
 [10358  8273  8829 10888]
 [15655 16862 12518 11722]
 [ 346

In [12]:
I = np.array([[0, 11, 0, 0],
            [22, 0, 0, 33],
            [0, 0, 0, 0],
            [0, 44, 0, 0]])
W = np.array([[2, 2],
            [1, 3],
            [3, 1],
            [1, 2]])
O = np.matmul(I,W)

In [8]:
O

array([[ 11,  33],
       [ 77, 110],
       [  0,   0],
       [ 44, 132]])

In [10]:
np.ones(O.shape, dtype =int)

array([[1, 1],
       [1, 1],
       [1, 1],
       [1, 1]])

In [5]:
val = np.array([11,22,33,44])
col = np.array([1,0,3,1])
row = np.array([0,1,3,3,4])
den = np.array([[2, 2],
            [1, 3],
            [3, 1],
            [1, 2]])

In [16]:
dataGen = data()
rowBin = dataGen.arrayToBinary(row)
colBin = dataGen.arrayToBinary(col)
valBin = dataGen.arrayToBinary(val)
denBin = dataGen.matrixToBinary(den)
rowAddr = pow(2,8)
colAddr = 2 * pow(2,8)
valAddr = 3 * pow(2,8)
denAddr = 4 * pow(2,8)
(_,x) = den.shape
(y,_) = I.shape
instGen = inst()
instr = ''
instr = instr + instGen.load(xsize = row.size, id = 'row', dram_offset = rowAddr, sram_offset = 0)
instr = instr + instGen.load(xsize = col.size, id = 'col', dram_offset = colAddr, sram_offset = 0)
instr = instr + instGen.load(xsize = val.size, id = 'val', dram_offset = valAddr, sram_offset = 0)
instr = instr + instGen.load(xsize = den.size, id = 'den', dram_offset = denAddr, sram_offset = 0)
instr = instr + instGen.spMM(ysize = y, xsize = x)
instCount = len(instr)/128
while(instCount%4 != 0):
    instr = instr + '0'*128
    instCount = len(instr)/128
dram = instr
while((len(dram)/8) != rowAddr):
    dram = dram + '0'*32
dram = dram + rowBin
while((len(dram)/8) != colAddr):
    dram = dram + '0'*32
dram = dram + colBin
while((len(dram)/8) != valAddr):
    dram = dram + '0'*32
dram = dram + valBin
while((len(dram)/8) != denAddr):
    dram = dram + '0'*32
dram = dram + denBin

In [21]:
from numpy import binary_repr
import numpy as np
import logging
import random

class inst:
    def __init__(self):
        logging.debug("Instruction generator created")
        
### Generates load instructions for GCN-Accelerator

    def load(self,  dep = '0000', id = 'col', sram_offset = 0, dram_offset = 0, xsize = 0, ysize = 0):
        idMap = {'col': '000','row': '001', 'val': '010', 'den': '011', 'out': '100'}
        op = '00'
        logging.debug(f"Creating load with \ndep = {dep}\nid = {id}\nsram offset = {sram_offset}\ndram offset = {dram_offset}\
        \nxsize = {xsize}\nysize = {ysize}")
        inst = op[::-1] + dep[::-1] + idMap.get(id)[::-1] + binary_repr(dram_offset, 32)[::-1]+ binary_repr(sram_offset, 16)[::-1]  
        logging.debug(f"inst[63:0] = {inst}")
        inst =  inst + binary_repr(xsize, 7)[::-1] + binary_repr(ysize, 0)[::-1]
        inst = inst + '0'*(128-len(inst))
        logging.debug(f"inst[127:0] = {inst}")
        return inst

    def spMM(self,  dep = '0000', sram_offset_col = 0, sram_offset_ptr = 0, sram_offset_den = 0, xsize = 0, ysize = 0):
        op = '10'
        logging.debug(f"Creating load with \ndep = {dep}\nid = {id}\nsram_offset_col = {sram_offset_col}\nsram_offset_ptr = {sram_offset_ptr}\
        \nsram_offset_den = {sram_offset_den}\nxsize = {xsize}\nysize = {ysize}")
        inst = op[::-1] + dep[::-1] + binary_repr(sram_offset_col, 16)[::-1] + binary_repr(sram_offset_ptr, 16)[::-1] + binary_repr(sram_offset_den, 16)[::-1]
        inst =  inst + binary_repr(xsize, 7)[::-1] + binary_repr(ysize, 7)[::-1]
        inst = inst + '0'*(128-len(inst))
        logging.debug(f"inst[127:0] = {inst}")
        return inst

    def randLoad(self, count = 1):
        instSeq = ""
        for num in range(count):
            instSeq = instSeq + "000000000"
            for i in range (119):
                instSeq = instSeq + str(random.randint(0,1))
        return instSeq
    
    def randSpMM(self, count = 1):
        instSeq = ""
        for num in range(count):
            instSeq = instSeq + "010000000"
            for i in range (119):
                instSeq = instSeq + str(random.randint(0,1))
        return instSeq


class data:
            
    ### Generates data for GCN-Accelerator
    def __init__(self):
        logging.debug("Data generator created")
    def matrixToBinary(self, A):
        string_A = ''
        for i in A:
            for j in i:
                string_A = string_A + binary_repr(j, 32)[::-1]
        return string_A
    def arrayToBinary(self, A):
        string_A = ''
        for i in A:
                string_A = string_A + binary_repr(i, 32)[::-1]
        return string_A



I = np.array([[0, 11, 0, 0],
            [22, 0, 0, 33],
            [0, 0, 0, 0],
            [0, 44, 0, 0]])
W = np.array([[2, 2],
            [1, 3],
            [3, 1],
            [1, 2]])
O = np.matmul(I,W)
val = np.array([11,22,33,44])
col = np.array([1,0,3,1])
row = np.array([0,1,3,3,4])
den = np.array([[2, 2],
            [1, 3],
            [3, 1],
            [1, 2]])
dataGen = data()
rowBin = dataGen.arrayToBinary(row)
colBin = dataGen.arrayToBinary(col)
valBin = dataGen.arrayToBinary(val)
denBin = dataGen.matrixToBinary(den)
rowAddr = pow(2,8)
colAddr = 2 * pow(2,8)
valAddr = 3 * pow(2,8)
denAddr = 4 * pow(2,8)
(_,x) = den.shape
(y,_) = I.shape
instGen = inst()
instr = ''
instr = instr + instGen.load(xsize = row.size, id = 'row', dram_offset = rowAddr, sram_offset = 0)
instr = instr + instGen.load(xsize = col.size, id = 'col', dram_offset = colAddr, sram_offset = 0)
instr = instr + instGen.load(xsize = val.size, id = 'val', dram_offset = valAddr, sram_offset = 0)
instr = instr + instGen.load(xsize = den.size, id = 'den', dram_offset = denAddr, sram_offset = 0)
instr = instr + instGen.spMM(ysize = y, xsize = x)
instCount = len(instr)/128
while(instCount%4 != 0):
    instr = instr + '0'*128
    instCount = len(instr)/128
dram = instr
while((len(dram)/8) != rowAddr):
    dram = dram + '0'*32
dram = dram + rowBin[::-1]
while((len(dram)/8) != colAddr):
    dram = dram + '0'*32
dram = dram + colBin[::-1]
while((len(dram)/8) != valAddr):
    dram = dram + '0'*32
dram = dram + valBin[::-1]
while((len(dram)/8) != denAddr):
    dram = dram + '0'*32
dram = dram + denBin[::-1]

In [29]:
hex(int(rowBin, 2))


'0x80000000c0000000c000000020000000'

In [13]:
hex(int(rowBin[::-1], 2))

5.0